In [5]:
import pandas as pd
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

stops = pd.read_csv("data/stops.txt")
trips = pd.read_csv("data/trips.txt")
stop_times = pd.read_csv("data/stop_times.txt")
routes = pd.read_csv("data/routes.txt")

C:\Users\balin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\balin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Which stops can be reached without switching rides?

In [1]:
def reachable_stops( depart_stop ):
    """
    Find all stops that has a matching substring with depart_stop
    and list all the reachable stops from them.
    """
    with driver.session() as session:

        response = session.run(f"""
        MATCH (s1: Stop)
        WHERE s1.stop_name CONTAINS "{depart_stop}"

        MATCH (s1)-[:STOPS_AT]-(t: Trip)-[:STOPS_AT]-(s2: Stop)
        RETURN DISTINCT s2.stop_name

        """)

        stop_names = [ record[0] for record in response]
    return stop_names


In [41]:
asd = reachable_stops("Móricz Zsigmond körtér")
len(asd)

439

# Which ride can be switched to from a given ride?

In [ ]:
# Pre-compute distances of stops which are close to each other


with driver.session() as session:
    session.run( \
"""MATCH (s1: Stop), (s2: Stop)

WITH point.distance(point( {x: s1.stop_lon, y: s1.stop_lat, crs: "WGS-84"}) ,
                    point( {x: s2.stop_lon, y: s2.stop_lat, crs: "WGS-84"})) as dist,
s1 as s1,
s2 as s2

WHERE dist < 600      //only create relationship between stops closer than 600m

MERGE (s1)-[:CLOSE_TO {d: dist}]-(s2)"""
)


In [8]:
def switch_from_ride(trip_id):
    with driver.session() as session:
        response = session.run( \
        """
        MATCH (t1: Trip{trip_id : "%s"})
        MATCH (t1)-[st1:STOPS_AT]-(: Stop)-[cls:CLOSE_TO]-(s2: Stop)-[st2:STOPS_AT]-(t2: Trip)
        WHERE st1.arrival_time < st2.departure_time    //it speeds up the calc a little


        WITH duration.inSeconds( st1.arrival_time, st2.departure_time ) as dur,
        t2 as t2,
        s2 as s2

        WHERE dur.seconds*1 > cls.d     // the stop must be in reachable distance
        and dur.seconds < 600          // make the switch in 10 minutes

        RETURN t2.trip_id, t2.route_id, s2.stop_name, dur.seconds
        """ % trip_id
            )

        rides = [{"trip_id": r[0], "route_id": r[1], "stop_name": r[2], "duration": r[3]} 
                    for r in response]
    
    return rides

In [9]:
switch_from_ride("C413899203")

[{'trip_id': 'C0713761',
  'route_id': 5400,
  'stop_name': 'Bikás park',
  'duration': 398},
 {'trip_id': 'C40537119',
  'route_id': '5400',
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C40535521',
  'route_id': '5400',
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C4053162',
  'route_id': '5400',
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C40538119',
  'route_id': '5400',
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C08506119',
  'route_id': 5400,
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C07137123',
  'route_id': 5400,
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C11578544',
  'route_id': 5400,
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C04430119',
  'route_id': 5400,
  'stop_name': 'Bikás park',
  'duration': 583},
 {'trip_id': 'C411611',
  'route_id': '9070',
  'stop_name': 'Bikás park M',
  'duration': 540},
 {'trip_id': 'C403811',
  'route_id': '907

In [5]:
stop_times[ stop_times.arrival_time != stop_times.departure_time].shape

(79326, 9)

- In ~99% (79.326 out of 6.441.309) of the time the arrival and departure time are the same. So I chose not to make distinct relationships for both.

- The preferred walking speed is 1.42 m/s, however only the aerial distance is available for now (the real distance is out of scope), so I chose the walking speed to be 1 m/s

- GTFS uses the WGS84 CRS, I used neo4j's spatial functions for calculating distance

(79326, 9)

0.012315198665364447